Construct a linear regression that takes RNA expression data as an input and outputs whether the patient has pediatric Crohn's disease or ulcerative colitis.

In [28]:
import pandas as pd
import numpy as np
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import MultiTaskLasso

In [27]:
'''
ONLY RUN THIS CELL IF WE WANT TO LIMIT NUMBER OF GENES FOR MODEL
'''
# Get the genes that we want to consider when running logistic regression
f = open("clusters.txt", "r").read()
clusters = eval(f)
print(clusters)
genes_to_use = clusters

['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2', 'NFYA', 'ENPP4', 'SEMA3F', 'CFTR', 'ANKIB1', 'LAP3', 'SNX11', 'M6PR', 'CYP26B1', 'CASP10', 'CFLAR', 'TFPI', 'PLXND1', 'CD38', 'RECQL', 'PDK4', 'CDC27', 'SKAP2', 'PRKAR2B', 'IBTK', 'LAMP2', 'TMEM132A', 'AP2B1', 'RALA', 'KDM7A', 'FARP2', 'NOS2', 'AC005747.1', 'CEACAM7', 'NADK', 'MMP25', 'SEC62', 'CD9', 'CD4', 'MRC2', 'PLAUR', 'DCN', 'TYROBP', 'CLCA1', 'CLCA4', 'ATP2C1', 'CD74', 'BIRC3', 'TYMP', 'VIM', 'CD44', 'SLAMF7', 'BTN3A1', 'TNFRSF1B', 'GRN', 'TIMP2', 'TNC', 'LCP2', 'HSPA5', 'HERPUD1', 'MXD1', 'VMP1', 'PKM', 'TFRC', 'ACTB', 'MMP2', 'LYZ', 'OLFM4', 'CEACAM5', 'COL1A1', 'SOD2', 'REG1A', 'STAT1', 'JCHAIN', 'PIGR', 'B2M', 'IGKC', 'IGHA1', 'IGHM']


In [19]:
'''
ONLY RUN THIS CELL IF WE WANT TO LIMIT NUMBER OF GENES FOR MODEL
'''
f = open("geneNames.txt", "r")
lines = [elem.strip() for elem in f.readlines()]
gene_inds_to_use = []
for elem in genes_to_use:
    gene_inds_to_use.append(lines.index(elem))
gene_inds_to_use

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 9,
 13,
 14,
 15,
 16,
 22,
 30,
 32,
 34,
 38,
 39,
 40,
 48,
 50,
 55,
 60,
 67,
 74,
 90,
 103,
 111,
 124,
 125,
 134,
 136,
 143,
 169,
 173,
 176,
 203,
 222,
 229,
 252,
 266,
 277,
 296,
 300,
 306,
 369,
 370,
 372,
 393,
 428,
 447,
 452,
 456,
 459,
 460,
 468,
 486,
 524,
 578,
 597,
 606,
 688,
 803,
 847,
 1006,
 1187,
 1317,
 1732,
 1876,
 2712,
 3099,
 3595,
 4005,
 4429,
 4432,
 6604,
 10883,
 11941,
 21828,
 22046,
 22050]

In [29]:
df = pd.read_csv('norm_data.tsv', sep = '\t')
df

,Unnamed: 0,SRR1782685,SRR1782686,SRR1782687,SRR1782688,SRR1782689,SRR1782690,SRR1782691,SRR1782692,SRR1782693,...,SRR1782997,SRR1782998,SRR1782999,SRR1783000,SRR1783001,SRR1783002,SRR1783003,SRR1783004,SRR1783005,SRR1783006
0,0,5.064846e-05,1.390220e-05,0.000086,4.830331e-05,2.264291e-05,5.634691e-05,5.001545e-05,5.633922e-05,7.515083e-05,...,6.160122e-05,3.568170e-05,4.330241e-05,5.397875e-05,7.586702e-05,0.000078,3.859068e-05,5.708651e-05,8.075028e-05,5.118729e-05
1,1,7.559472e-07,0.000000e+00,0.000001,0.000000e+00,0.000000e+00,7.087662e-07,1.069849e-06,0.000000e+00,2.994057e-07,...,7.228305e-07,2.885313e-07,0.000000e+00,2.578603e-07,7.848312e-07,0.000000,1.628299e-07,0.000000e+00,6.117446e-07,0.000000e+00
2,2,2.570220e-05,2.585549e-05,0.000031,2.320247e-05,3.402981e-05,3.437516e-05,2.888593e-05,3.082236e-05,3.473106e-05,...,3.059982e-05,3.202697e-05,3.007112e-05,3.231849e-05,3.073922e-05,0.000030,3.489987e-05,2.801088e-05,3.476748e-05,3.348596e-05
3,3,1.565891e-05,1.390220e-05,0.000016,1.750731e-05,2.002523e-05,1.346656e-05,2.353668e-05,1.717967e-05,2.095840e-05,...,2.120303e-05,2.115896e-05,2.135049e-05,1.839403e-05,2.014400e-05,0.000012,1.915965e-05,1.703586e-05,1.570144e-05,1.420616e-05
4,4,3.779736e-06,5.586865e-06,0.000002,4.113164e-06,4.842703e-06,6.378896e-06,6.686557e-06,5.305487e-06,3.592868e-06,...,6.746418e-06,5.578271e-06,5.713512e-06,4.813392e-06,7.717507e-06,0.000004,6.187535e-06,4.586577e-06,3.364595e-06,2.705936e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65212,65212,0.000000e+00,1.299271e-07,0.000000,6.327945e-07,1.308839e-07,0.000000e+00,5.349246e-07,0.000000e+00,2.994057e-07,...,1.606290e-07,0.000000e+00,1.503556e-07,0.000000e+00,2.616104e-07,0.000000,1.628299e-07,0.000000e+00,2.039149e-07,1.127473e-07
65213,65213,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
65214,65214,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
65215,65215,0.000000e+00,0.000000e+00,0.000000,1.054658e-07,0.000000e+00,0.000000e+00,2.674623e-07,2.526423e-07,2.994057e-07,...,8.031450e-08,9.617709e-08,0.000000e+00,1.719069e-07,1.308052e-07,0.000000,5.427662e-08,8.190317e-08,0.000000e+00,0.000000e+00


In [30]:
#Get a List of the Samples
samples = df.columns[1:]
samples

Index(['SRR1782685', 'SRR1782686', 'SRR1782687', 'SRR1782688', 'SRR1782689',
       'SRR1782690', 'SRR1782691', 'SRR1782692', 'SRR1782693', 'SRR1782694',
       ...
       'SRR1782997', 'SRR1782998', 'SRR1782999', 'SRR1783000', 'SRR1783001',
       'SRR1783002', 'SRR1783003', 'SRR1783004', 'SRR1783005', 'SRR1783006'],
      dtype='object', length=322)

In [1]:
#creat input matrix
df.drop(df.columns[0], axis = 1, inplace = True)
expression_by_gene = df.to_numpy()    #genes are rows, columns are samples
'''
UNCOMMENT FOLLOWING TWO LINES IF WE WANT TO LIMIT NUMBER OF GENES FOR MODEL
'''
#if gene_inds_to_use:
#    expression_by_gene = expression_by_gene[gene_inds_to_use,:]

expression_by_sample = np.transpose(expression_by_gene)    #samples are rows, genes are columns
expression_by_sample

NameError: name 'df' is not defined

In [32]:
#create output matrix
f = open('classifications.txt', 'r').read()
samples_to_label = eval(f)
print(len(samples_to_label))
output = []

for elem in samples:
    output.append(samples_to_label[elem])
output = np.array(output)

322


In [24]:
def run_regression():
    x_train, x_test, y_train, y_test = train_test_split(expression_by_sample, output, test_size=0.3)
    model = LogisticRegression(class_weight = 'balanced', multi_class='multinomial', solver='newton-cg')
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return accuracy

In [33]:
def run_lasso():
    x_train, x_test, y_train, y_test = train_test_split(expression_by_sample, output, test_size=0.3)
    clf = LogisticRegression(penalty='l1', class_weight = 'balanced', multi_class='multinomial', solver = 'saga')
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return accuracy

In [34]:
total = 0
num_reps = 1000
for i in range(num_reps):
    acc = run_lasso()
    #acc = run_regression()
    total += acc
accuracy = total/num_reps
accuracy

0.36755670103092797

In [35]:
f = open("log_reg_accuracy.txt", "a+")
#f.write("Genes used: "+str(genes_used))
f.write("Genes used: all \n")
f.write("Regularization: l1\n")
f.write(str(accuracy)+"\n")
f.close()